In [5]:
#!/usr/bin/env python
# coding: utf-8

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.axes as maxes

#import cartopy as cart

#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#from mpl_toolkits.axes_grid1 import make_axes_locatable

import csv
import math

import xarray as xr

import great_circle_calculator.great_circle_calculator as gcc
from great_circle_calculator.great_circle_calculator import distance_between_points

from datetime import datetime,timedelta

dirin="/Users/krishthe1nonly/Downloads/" 

day=datetime(2023,1,10)

#--find dates for previous day and next day
day_prev =day+timedelta(days=-1)
day_next =day+timedelta(days=1)
#
#--filenames
day_prev_file =str(day_prev.year) +'-'+str(day_prev.month).zfill(2) +'-'+str(day_prev.day).zfill(2)
day_file      =str(day.year)+'-'+str(day.month).zfill(2)+'-'+str(day.day).zfill(2)
day_next_file =str(day_next.year) +'-'+str(day_next.month).zfill(2) +'-'+str(day_next.day).zfill(2)
#
#--open FlightRadar24 file for selected day
print('file=',dirin+day_file+'.csv')
flights_fromday=pd.read_csv(dirin+day_file+'.csv', keep_default_na=False,\
                names=['address','altitude','callsign','date','dest_iata',\
                       'dest_icao','equipment','event','flight','flight_id','latitude',\
                       'longitude','operator','orig_iata','orig_icao','registration','time'])
#
#--consider flight_ids of the day but removing duplicates
flight_ids_fromday=list(set(flights_fromday.flight_id.values))
#
#--open FlightRadar24 file for previous day
print('prev file=',dirin+day_prev_file+'.csv')
flights_prevday=pd.read_csv(dirin+day_prev_file+'.csv', keep_default_na=False,\
                names=['address','altitude','callsign','date','dest_iata',\
                       'dest_icao','equipment','event','flight','flight_id','latitude',\
                       'longitude','operator','orig_iata','orig_icao','registration','time'])
#--remove flights if flight_id was already in previous day
flight_ids_prevday=list(set(flights_prevday.flight_id.values))
flights_fromday=flights_fromday[~flights_fromday.flight_id.isin(flight_ids_prevday)]
#
#--next day for overnight files
print('next file=',dirin+day_next_file+'.csv')
flights_nextday=pd.read_csv(dirin+day_next_file+'.csv', keep_default_na=False,\
                names=['address','altitude','callsign','date','dest_iata',\
                       'dest_icao','equipment','event','flight','flight_id','latitude',\
                       'longitude','operator','orig_iata','orig_icao','registration','time'])
#--keep flights from next day if they occur overnight
flights_nextday=flights_nextday[flights_nextday.flight_id.isin(flight_ids_fromday)]
#--lump flights from day and next day
flights=pd.concat([flights_fromday,flights_nextday], axis=0, ignore_index=True)

#===================================================== STEP 17 ====================================================================

flights = flights.drop(['altitude','flight', 'operator','registration'], axis =1)

# required_events = {'cruising', 'descent'}

# def set_req_events(flights, required_events):
#   flight_ids = flights['flight_id']

#   cruising_flights = flights[flights['event'].isin({'cruising'})]
#   cruising_ids = cruising_flights['flight_id'].unique()

#   descent_flights = flights[flights['event'].isin({'descent'})]
#   descent_ids = descent_flights['flight_id'].unique()

#   has_both_events = [id for id in cruising_ids if id in descent_ids]
#   both_eves = []
#   for flight_id in cruising_ids:
#     if flight_id in descent_ids:
#       both_eves.append(flight_id)
#   assert both_eves == has_both_events
#   return has_both_events

# has_both_events = set_req_events(flights, required_events)
# flights.set_index('flight_id', inplace=True)

# def get_long_lat(flights, flight_id):
#     current_fID = flights.loc[flight_id]
  
#     long1 = float(current_fID[current_fID['event']=='cruising']['longitude'])
#     lat1 = float(current_fID[current_fID['event']=='cruising']['latitude'])
    
#     long2 = float(current_fID[current_fID['event']=='descent']['longitude'])
#     lat2 = float(current_fID[current_fID['event']=='descent']['latitude'])
    
#     point1 = (long1, lat1) 
#     point2 = (long2, lat2) 
#     return point1, point2

# def calc_distance(point1, point2): 
#   #calculates the distance between the two points from the dataframe in kilometers and miles 
#   dist_km = distance_between_points(point1, point2, unit='kilometers', haversine=True)
#   dist_miles = dist_km*0.6213711922
#   print(dist_km)
#   return dist_km, dist_miles

# def calc_time_and_speed(flights, flight_id, dist_km, dist_miles):
#   current_fID = flights.loc[flight_id]
#   current_fID['date-time'] = pd.to_datetime(current_fID['date'] + " " + current_fID['time'])

#   time1 = current_fID[current_fID['event']=='cruising']['date-time']
#   time2 = current_fID[current_fID['event']=='descent']['date-time']
  
#   time_deltah = time2.iloc[0] - time1.iloc[0]
#   if time_deltah.days > 0:
#     print("WARNING: Flight is too long")
#   time_diff_int = time_deltah.seconds/3600

# #calculate average speed
#   avg_speed_kmh = dist_km/time_diff_int
#   avg_speed_mph = dist_miles/time_diff_int
#    #prints each result and formats
#   print(f"The distance between these points is {dist_km:.2f} km or {dist_miles:.2f} miles.")
#   print(f"The average speed of this aircraft was {avg_speed_kmh:.2f} km/h or {avg_speed_mph:.2f} mph.")
#   print("=============================")
#   return avg_speed_kmh

# histogram_distance = []
# histogram_speed = []

# #for loop that runs all defined functions for each flight id in the has_both_events Pandas series
# for flight_id in has_both_events:
#   current_fID = flights.loc[flight_id]
#   if len(current_fID[current_fID['event']=='cruising'].index) > 1: break
#   if len(current_fID[current_fID['event']=='descent'].index) > 1: break
#   long_lat1, long_lat2 = get_long_lat(flights, flight_id)
#   calculated_dist_km, calculated_dist_mi = calc_distance(long_lat1, long_lat2)
#   print(calculated_dist_km)
#   calculated_ts = calc_time_and_speed(flights, flight_id, calculated_dist_km, calculated_dist_mi)
#   histogram_distance.append(calculated_dist_km)
#   histogram_speed.append(calculated_ts)

# graphit = True
# if graphit:
#     graph_colors = ['lightblue', 'blue', 'purple', 'red', 'black', 'green', 'orange', 'grey']

#     # Plot histogram of distances
#     plt.hist(histogram_distance, bins=20, color='blue', edgecolor='black')
#     plt.yscale('log')
#     plt.xlabel("Distance (km)")
#     plt.ylabel("Number of Flights")
#     plt.title("Distribution of Flight Distances")
#     plt.show()

#     plt.hist(histogram_speed, bins=20, color='green', edgecolor='black')
#     plt.yscale('log')
#     plt.xlabel("Average Speed (km/h)")
#     plt.ylabel("Number of Flights")
#     plt.title("Distribution of Average Speeds")
#     plt.savefig()
#     plt.show()
#===============================================================================================================================

#===================================================== STEP 18 & 19 ====================================================================

# Correct the groupby call to use a single list of column names
grouped_column_flights = flights.groupby(['address', 'callsign', 'dest_iata', 'dest_icao', 'orig_iata', 'orig_icao', 'equipment'])

# Get the size of each group and its index
sized_indexed = grouped_column_flights.size().index

# Identify and filter groups with missing information
missing_info_groups = [idx for idx in sized_indexed if any(pd.isnull(x) or x == '' for x in idx)]

# Display the groups with missing information
print("Groups with missing information:")
for idx in missing_info_groups:
    print(idx)





lat     = [80,30,-0.1,17,30]
lon     = [100,-100,0.,77.,-120]
color   = [2,-1,1.,-1.4,-2]  # could be bias
size    = [30,200,100,40,15]   # could be coef correl
size    = [30,30 ,30 ,30,30]
sizelab = [10,200]     # should match values in size
titre   = 'TEST titre'
colpal  = 'jet' #'PiYG'
mincol  = -1. # should match values in color
maxcol  = 1.  # should match values in color
sizetit = ' test lab' #title for the size of the label if needed (value is anyway display)
fileoutpng = '/Users/krishthe1nonly/Downloads/mapforcoding.jpeg'
colorbar_labsize = 23
ncols = 10  # should be 0 if you want a non-discrete colorpalette

#plot_station(fileoutpng,lat,lon,color,size,
             #sizelab,sizetit=sizetit,colbar_tit=titre,
             #palette=colpal,mincol=mincol,maxcol=maxcol,n_colors=ncols,colorbar_labsize=colorbar_labsize,reverse_colpal='no')






file= /Users/krishthe1nonly/Downloads/2023-01-10.csv
prev file= /Users/krishthe1nonly/Downloads/2023-01-09.csv
next file= /Users/krishthe1nonly/Downloads/2023-01-11.csv
Groups with missing information:
('000000', '', '', '', '', '', '')
('000000', '', '', '', 'ADD', 'HAAB', '')
('000000', '', '', '', 'BOG', 'SKBO', '')
('000000', '', '', '', 'CEB', 'RPVM', '')
('000000', '', '', '', 'CGN', 'EDDK', '')
('000000', '', '', '', 'DEL', 'VIDP', '')
('000000', '', '', '', 'DME', 'UUDD', '')
('000000', '', '', '', 'DOH', 'OTHH', '')
('000000', '', '', '', 'EYW', 'KEYW', '')
('000000', '', '', '', 'GDN', 'EPGD', '')
('000000', '', '', '', 'GYY', 'KGYY', '')
('000000', '', '', '', 'HKG', 'VHHH', '')
('000000', '', '', '', 'ISL', 'LTBA', '')
('000000', '', '', '', 'LYX', 'EGMD', '')
('000000', '', '', '', 'MGQ', 'HCMM', '')
('000000', '', '', '', 'MUC', 'EDDM', '')
('000000', '', '', '', 'NCY', 'LFLP', '')
('000000', '', '', '', 'OMD', 'FYOG', '')
('000000', '', '', '', 'OSD', 'ESNZ', '')
('00000